# Import Libraries

In [1]:
import numpy as np
import mxnet as mx
from mxnet import autograd
from mxnet import gluon
from mxnet import nd



In [2]:
ctx = mx.cpu()

# Define a function to encode the integer to its binary representation

In [3]:
def binary_encode(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])

# Define a function to label the data

In [4]:
def fizz_buzz_encode(i):
    if   i % 15 == 0: 
        return 0
    elif i % 5  == 0: 
        return 1
    elif i % 3  == 0: 
        return 2
    else:             
        return 3

# Create the Numpy NdArray for traing and test data

In [5]:
MAX_NUMBER = 20000
NUM_DIGITS = np.log2(MAX_NUMBER).astype(np.int)+1
trainX = np.array([binary_encode(i, NUM_DIGITS) for i in range(101, np.int(MAX_NUMBER/2))])
trainY = np.array([fizz_buzz_encode(i)          for i in range(101, np.int(MAX_NUMBER/2))])
valX = np.array([binary_encode(i, NUM_DIGITS) for i in range(np.int(MAX_NUMBER/2), MAX_NUMBER)])
valY = np.array([fizz_buzz_encode(i)          for i in range(np.int(MAX_NUMBER/2), MAX_NUMBER)])
testX = np.array([binary_encode(i, NUM_DIGITS) for i in range(1, 101)])
testY = np.array([fizz_buzz_encode(i)          for i in range(1, 101)])

# Create mxnet NDarrayiter for training and test data

In [6]:
batch_size = 100
num_inputs = NUM_DIGITS
num_outputs = 4
train_data = mx.io.NDArrayIter(trainX, trainY,
                               batch_size, shuffle=True)
val_data = mx.io.NDArrayIter(valX, valY,
                               batch_size, shuffle=True)
test_data = mx.io.NDArrayIter(testX, testY,
                              batch_size, shuffle=False)

# Define the Gluon Sequestial Model

In [7]:
num_hidden = 256
net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Dense(num_inputs, activation="relu"))
    net.add(gluon.nn.Dense(num_hidden, activation="relu"))
    net.add(gluon.nn.Dense(num_outputs))

# Initialize Parameter

In [8]:
net.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)

# Softmax Cross Entropy Loss

In [9]:
loss = gluon.loss.SoftmaxCrossEntropyLoss()

# Stochastic Gradient Descent Optimizer

In [10]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': .1,'momentum':0.9})



# Define the function for evaluating Accuracy

In [11]:
def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    data_iterator.reset()
    for i, batch in enumerate(data_iterator):
        data = batch.data[0].as_in_context(ctx).reshape((-1, num_inputs))
        label = batch.label[0].as_in_context(ctx)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

# Lets Train the MLP model 

In [12]:
epochs = 100
moving_loss = 0.

for e in range(epochs):
    train_data.reset()
    for i, batch in enumerate(train_data):
        data = batch.data[0].as_in_context(ctx).reshape((-1, num_inputs))
        label = batch.label[0].as_in_context(ctx)
        with autograd.record():
            output = net(data)
            cross_entropy = loss(output, label)
            cross_entropy.backward()
        trainer.step(data.shape[0])
        if i == 0:
            moving_loss = nd.mean(cross_entropy).asscalar()
        else:
            moving_loss = .99 * moving_loss + .01 * nd.mean(cross_entropy).asscalar()

    val_accuracy = evaluate_accuracy(val_data, net)
    train_accuracy = evaluate_accuracy(train_data, net)
    print("Epoch %s. Loss: %s, Train_acc %s, Val_acc %s" %
          (e, moving_loss, train_accuracy, val_accuracy))

Epoch 0. Loss: 1.23673449719, Train_acc 0.533434343434, Val_acc 0.5334
Epoch 1. Loss: 1.13576843941, Train_acc 0.533434343434, Val_acc 0.5334
Epoch 2. Loss: 1.13470083972, Train_acc 0.533434343434, Val_acc 0.5334
Epoch 3. Loss: 1.13372674407, Train_acc 0.533434343434, Val_acc 0.5334
Epoch 4. Loss: 1.13286777237, Train_acc 0.533434343434, Val_acc 0.5334
Epoch 5. Loss: 1.13115855296, Train_acc 0.533434343434, Val_acc 0.5334
Epoch 6. Loss: 1.12697293319, Train_acc 0.533434343434, Val_acc 0.5334
Epoch 7. Loss: 1.10770986514, Train_acc 0.526464646465, Val_acc 0.5327
Epoch 8. Loss: 1.02883823208, Train_acc 0.635757575758, Val_acc 0.5602
Epoch 9. Loss: 0.853434077768, Train_acc 0.678282828283, Val_acc 0.5504
Epoch 10. Loss: 0.665760026595, Train_acc 0.74404040404, Val_acc 0.5515
Epoch 11. Loss: 0.592008043265, Train_acc 0.780404040404, Val_acc 0.5956
Epoch 12. Loss: 0.549476172342, Train_acc 0.807575757576, Val_acc 0.6096
Epoch 13. Loss: 0.545677343166, Train_acc 0.801616161616, Val_acc 0.602

In [14]:
def fizz_buzz(i, prediction):
    if prediction == 0:
        return "fizzbuzz"
    elif prediction == 1:
        return "buzz"
    elif prediction == 2:
        return "fizz"
    else:
        return str(i)
    
test_data.reset()
for i, batch in enumerate(test_data):
    data = batch.data[0].as_in_context(ctx).reshape((-1, num_inputs))
    label = batch.label[0].as_in_context(ctx)
    output = net(data)
    predictions = nd.argmax(output, axis=1)
    print(predictions)
    output = np.vectorize(fizz_buzz)(np.arange(1, 101), predictions.asnumpy().astype(np.int))
    print(output)
    acc = mx.metric.Accuracy()
    acc.update(preds=predictions, labels=label)
    print(acc.get()[1])


[ 3.  3.  2.  3.  1.  2.  3.  3.  2.  1.  3.  2.  3.  1.  0.  3.  3.  2.
  3.  1.  3.  3.  3.  2.  1.  3.  2.  3.  3.  0.  3.  3.  2.  3.  1.  2.
  3.  3.  2.  1.  3.  2.  3.  1.  0.  3.  3.  2.  3.  1.  2.  3.  3.  2.
  3.  3.  2.  3.  3.  0.  3.  3.  2.  3.  1.  2.  3.  3.  3.  1.  3.  2.
  3.  3.  0.  3.  3.  2.  3.  1.  3.  3.  3.  3.  1.  3.  3.  3.  3.  0.
  3.  3.  3.  3.  1.  2.  3.  3.  2.  1.]
<NDArray 100 @cpu(0)>
['1' '2' 'fizz' '4' 'buzz' 'fizz' '7' '8' 'fizz' 'buzz' '11' 'fizz' '13'
 'buzz' 'fizzbuzz' '16' '17' 'fizz' '19' 'buzz' '21' '22' '23' 'fizz'
 'buzz' '26' 'fizz' '28' '29' 'fizzbuzz' '31' '32' 'fizz' '34' 'buzz'
 'fizz' '37' '38' 'fizz' 'buzz' '41' 'fizz' '43' 'buzz' 'fizzbuzz' '46'
 '47' 'fizz' '49' 'buzz' 'fizz' '52' '53' 'fizz' '55' '56' 'fizz' '58' '59'
 'fizzbuzz' '61' '62' 'fizz' '64' 'buzz' 'fizz' '67' '68' '69' 'buzz' '71'
 'fizz' '73' '74' 'fizzbuzz' '76' '77' 'fizz' '79' 'buzz' '81' '82' '83'
 '84' 'buzz' '86' '87' '88' '89' 'fizzbuzz' '91' '92' '93' '9